In [1]:
import elasticsearch
import datetime
import pandas
import json
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# setup some initial vars
ES_HOST = {"host" : "128.173.145.158", "port" : 9200}
INDEX_NAME = 'dst'
TYPE_NAME = 'dst'
inpDstFile = "../dst_out_file.csv"

In [3]:
# read data into a pandas DF
dstDF = pandas.read_csv(inpDstFile, sep=' ',\
                            parse_dates=["dst_date"], infer_datetime_format=True)
dstRecsStr = dstDF.to_json(orient="records")
dstRecs = json.loads(dstRecsStr)

In [7]:
es = elasticsearch.Elasticsearch(hosts = [ES_HOST])
res = es.indices.create(index = INDEX_NAME)

In [8]:
res = es.bulk(index = INDEX_NAME, body = dstRecs, refresh = True)
print res

RequestError: TransportError(400, u'illegal_argument_exception', u'Malformed action/metadata line [1], expected START_OBJECT or END_OBJECT but found [VALUE_NUMBER]')